In [2]:
import pandas
import researchpy as rp
import seaborn as sns

import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp


# Analysis <font color=blue>after </font> combining RF, DT and SVM into 1C

In [3]:
df = pandas.read_excel(r"C:\Users\HassanEldeeb\Documents\GitHub\AutoMLBenchmarking\logs_search_space/atmout.xlsx")
df = df[['dataset', 'time_budget', 'methods', 'f1score']]
df.methods = df.methods.replace("['logreg', 'svm', 'sgd', 'dt', 'et', 'rf', 'gnb', 'mnb', 'bnb', 'gp', 'pa', 'knn', 'mlp', 'ada']", "fc")
df.methods = df.methods.replace("['rf', 'dt', 'svm']", "3c")
df.methods = df.methods.replace("['logreg', 'dt', 'knn']", "def")
df.methods = df.methods.replace("['svm']", "1c")
df.methods = df.methods.replace("['dt']", "1c")
df.methods = df.methods.replace("['rf']", "1c")
fsIs_b = ['vowel', 'openml_phpJNxH0q', 'dataset_31_credit-g', 'dataset_40_sonar']
fsIs_m = ['solar-flare_1', 'wine-quality-red', 'dataset_39_ecoli', 'synthetic_control']
fsIl_b = ['AirlinesCodrnaAdult', 'MagicTelescope', 'electricity-normalized', 'phpmPOD5A']
fsIl_m = ['pokerhand-normalized', 'eye_movements', 'avila-tr']
flIs_b = ['audiology', 'arrhythmia', 'AP_Breast_Lung', 'AP_Omentum_Ovary']
flIs_m = ['Amazon', 'umistfacescropped', 'phpGUrE90']
flIl_b = ['gina_agnostic', 'hiva_agnostic', 'phpZrCzJR', 'phprAeXmK']
flIl_m = ['KDDCup99', 'connect-4', 'dataset_60_waveform-5000', 'dataset_186_satimage']
df_binary = df[df.dataset.isin(fsIs_b) | df.dataset.isin(fsIl_b) | df.dataset.isin(flIs_b) | df.dataset.isin(flIl_b)]
df_multi = df[df.dataset.isin(fsIs_m) | df.dataset.isin(fsIl_m) | df.dataset.isin(flIs_m) | df.dataset.isin(flIl_m)]
df = df_binary
#df_multi = df[df[(((df.dataset in fsIs_m) | (df.dataset in fsIl_m)) | ((df.dataset in flIs_m) | (df.dataset in flIl_m)))].index]
#df.drop(df[(df.methods=='fc') & ((df.time_budget==30) | (df.time_budget==10))].index, inplace=True)
#df.drop(df[(df.methods=='3c') & ((df.time_budget==60) | (df.time_budget==10))].index, inplace=True)
df.shape

(581, 4)

In [4]:
rp.summary_cont(df['f1score'])

,Variable,N,Mean,SD,SE,95% Conf.,Interval
0,f1score,581.0,0.818827,0.20872,0.008659,0.80182,0.835834


In [5]:
rp.summary_cont(df.groupby(['time_budget']))['f1score']

,N,Mean,SD,SE,95% Conf.,Interval
time_budget,,,,,,
10,161,0.823724,0.197346,0.015553,0.793240,0.854208
30,223,0.828943,0.192694,0.012904,0.803652,0.854235
60,197,0.803375,0.233856,0.016662,0.770718,0.836031


In [6]:
rp.summary_cont(df.groupby(['methods']))['f1score']

,N,Mean,SD,SE,95% Conf.,Interval
methods,,,,,,
1c,317,0.801564,0.214675,0.012057,0.777931,0.825196
3c,63,0.841806,0.215475,0.027147,0.788597,0.895015
def,70,0.851325,0.176365,0.021080,0.810009,0.892641
fc,131,0.832187,0.204831,0.017896,0.797110,0.867264


In [7]:
rp.summary_cont(df.groupby(['time_budget', 'methods']))['f1score']

N      Mean        SD        SE  95% Conf.  Interval
time_budget methods                                                        
10          1c        83  0.784026  0.224429  0.024634   0.735743  0.832309
            3c        12  0.837359  0.193220  0.055778   0.728034  0.946683
            def       20  0.872689  0.142873  0.031947   0.810073  0.935306
            fc        46  0.870507  0.150621  0.022208   0.826979  0.914034
30          1c       127  0.820008  0.197673  0.017541   0.785629  0.854388
            3c        23  0.870051  0.149337  0.031139   0.809018  0.931083
            def       21  0.873640  0.114389  0.024962   0.824715  0.922565
            fc        52  0.814532  0.219875  0.030491   0.754769  0.874295
60          1c       107  0.793275  0.226400  0.021887   0.750377  0.836173
            3c        28  0.820511  0.268469  0.050736   0.721068  0.919953
            def       29  0.820431  0.227518  0.042249   0.737623  0.903239
            fc        33  0.806592  0.241245  0.041995   0.724281  0.888903

In [8]:
    # Fits the model with the interaction term
    # This will also automatically include the main effects for each factor
    model = ols('f1score ~ C(time_budget)*C(methods)', df).fit()

    # Seeing if the overall model is significant
    print(f"Overall model F({model.df_model: .0f},{model.df_resid: .0f}) = {model.fvalue: .3f}, p = {model.f_pvalue: .4f}")

Overall model F( 11, 569) =  1.012, p =  0.4336


In [9]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                f1score   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.012
Date:                Fri, 01 Nov 2019   Prob (F-statistic):              0.434
Time:                        15:24:50   Log-Likelihood:                 92.017
No. Observations:                 581   AIC:                            -160.0
Df Residuals:                     569   BIC:                            -107.7
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
==========================================================================================================
                                             coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------
Intercept                                  0.7840      0.023     34.226      0.000       0.739       0.829
C(time_budget)[T.30]                       0.0360      0.029      1.222      0.222      -0.022       0.094
C(time_budget)[T.60]                       0.0092      0.031      0.303      0.762      -0.051       0.069
C(methods)[T.3c]                           0.0533      0.064      0.827      0.408      -0.073       0.180
C(methods)[T.def]                          0.0887      0.052      1.706      0.089      -0.013       0.191
C(methods)[T.fc]                           0.0865      0.038      2.254      0.025       0.011       0.162
C(time_budget)[T.30]:C(methods)[T.3c]     -0.0033      0.080     -0.041      0.967      -0.160       0.154
C(time_budget)[T.60]:C(methods)[T.3c]     -0.0261      0.078     -0.334      0.739      -0.180       0.128
C(time_budget)[T.30]:C(methods)[T.def]    -0.0350      0.072     -0.490      0.625      -0.176       0.106
C(time_budget)[T.60]:C(methods)[T.def]    -0.0615      0.068     -0.906      0.365      -0.195       0.072
C(time_budget)[T.30]:C(methods)[T.fc]     -0.0920      0.051     -1.786      0.075      -0.193       0.009
C(time_budget)[T.60]:C(methods)[T.fc]     -0.0732      0.057     -1.294      0.196      -0.184       0.038
==============================================================================
Omnibus:                      160.091   Durbin-Watson:                   2.215
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              315.714
Skew:                          -1.575   Prob(JB):                     2.78e-69
Kurtosis:                       4.766   Cond. No.                         17.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [10]:
# Creates the ANOVA table
res = sm.stats.anova_lm(model, typ= 2)
res

,sum_sq,df,F,PR(>F)
C(time_budget),0.085776,2.0,0.984722,0.374179
C(methods),0.237103,3.0,1.814643,0.143329
C(time_budget):C(methods),0.174231,6.0,0.666729,0.676627
Residual,24.782014,569.0,NaN,NaN


In [11]:
# Fits the model
model2 = ols('f1score ~ C(time_budget)+ C(methods)', df).fit()

print(f"Overall model F({model2.df_model: .0f},{model2.df_resid: .0f}) = {model2.fvalue: .3f}, p = {model2.f_pvalue: .4f}")

Overall model F( 5, 575) =  1.432, p =  0.2107


In [12]:
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                f1score   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     1.432
Date:                Fri, 01 Nov 2019   Prob (F-statistic):              0.211
Time:                        15:25:01   Log-Likelihood:                 89.982
No. Observations:                 581   AIC:                            -168.0
Df Residuals:                     575   BIC:                            -141.8
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                0.8058      0.019     43.551      0.000       0.769       0.842
C(time_budget)[T.30]     0.0071      0.022      0.328      0.743      -0.035       0.050
C(time_budget)[T.60]    -0.0210      0.022     -0.942      0.347      -0.065       0.023
C(methods)[T.3c]         0.0427      0.029      1.484      0.138      -0.014       0.099
C(methods)[T.def]        0.0521      0.028      1.889      0.059      -0.002       0.106
C(methods)[T.fc]         0.0288      0.022      1.328      0.185      -0.014       0.072
==============================================================================
Omnibus:                      164.876   Durbin-Watson:                   2.199
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              333.005
Skew:                          -1.605   Prob(JB):                     4.89e-73
Kurtosis:                       4.859   Cond. No.                         4.57
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [13]:
# Creates the ANOVA table
res2 = sm.stats.anova_lm(model2, typ= 2)
res2

,sum_sq,df,F,PR(>F)
C(time_budget),0.085776,2.0,0.988158,0.372893
C(methods),0.237103,3.0,1.820976,0.142158
Residual,24.956245,575.0,NaN,NaN


In [14]:
# Calculating effect size
def anova_table(aov):
    aov['mean_sq'] = aov[:]['sum_sq']/aov[:]['df']
    
    aov['eta_sq'] = aov[:-1]['sum_sq']/sum(aov['sum_sq'])
    
    aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df']*aov['mean_sq'][-1]))/(sum(aov['sum_sq'])+aov['mean_sq'][-1])
    
    cols = ['sum_sq', 'mean_sq', 'df', 'F', 'PR(>F)', 'eta_sq', 'omega_sq']
    aov = aov[cols]
    return aov

anova_table(res2)

,sum_sq,mean_sq,df,F,PR(>F),eta_sq,omega_sq
C(time_budget),0.085776,0.042888,2.0,0.988158,0.372893,0.003393,-0.000041
C(methods),0.237103,0.079034,3.0,1.820976,0.142158,0.009379,0.004221
Residual,24.956245,0.043402,575.0,NaN,NaN,NaN,NaN


In [15]:
mc = statsmodels.stats.multicomp.MultiComparison(df['f1score'], df['methods'])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
    1c     3c   0.0402 0.4995 -0.0338 0.1143  False
    1c    def   0.0498 0.2704 -0.0211 0.1206  False
    1c     fc   0.0306 0.4906 -0.0251 0.0864  False
    3c    def   0.0095    0.9 -0.0837 0.1027  False
    3c     fc  -0.0096    0.9 -0.0919 0.0727  False
   def     fc  -0.0191    0.9 -0.0986 0.0603  False
---------------------------------------------------


In [16]:
mc = statsmodels.stats.multicomp.MultiComparison(df['f1score'], df['time_budget'])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
    10     30   0.0052    0.9 -0.0455  0.056  False
    10     60  -0.0203  0.619 -0.0725 0.0318  False
    30     60  -0.0256 0.4246 -0.0735 0.0224  False
---------------------------------------------------


# Analysis <font color=blue>without </font> combining RF, DT and SVM into 1C

In [30]:
df = pandas.read_excel(r"C:\Users\HassanEldeeb\Documents\GitHub\AutoMLBenchmarking\logs_search_space/atmout.xlsx")
df = df[['dataset', 'time_budget', 'methods', 'f1score']]
df.methods = df.methods.replace("['logreg', 'svm', 'sgd', 'dt', 'et', 'rf', 'gnb', 'mnb', 'bnb', 'gp', 'pa', 'knn', 'mlp', 'ada']", "fc")
df.methods = df.methods.replace("['rf', 'dt', 'svm']", "3c")
df.methods = df.methods.replace("['logreg', 'dt', 'knn']", "def")
df.methods = df.methods.replace("['svm']", "svm")
df.methods = df.methods.replace("['dt']", "dt")
df.methods = df.methods.replace("['rf']", "rf")
fsIs_b = ['vowel', 'openml_phpJNxH0q', 'dataset_31_credit-g', 'dataset_40_sonar']
fsIs_m = ['solar-flare_1', 'wine-quality-red', 'dataset_39_ecoli', 'synthetic_control']
fsIl_b = ['AirlinesCodrnaAdult', 'MagicTelescope', 'electricity-normalized', 'phpmPOD5A']
fsIl_m = ['pokerhand-normalized', 'eye_movements', 'avila-tr']
flIs_b = ['audiology', 'arrhythmia', 'AP_Breast_Lung', 'AP_Omentum_Ovary']
flIs_m = ['Amazon', 'umistfacescropped', 'phpGUrE90']
flIl_b = ['gina_agnostic', 'hiva_agnostic', 'phpZrCzJR', 'phprAeXmK']
flIl_m = ['KDDCup99', 'connect-4', 'dataset_60_waveform-5000', 'dataset_186_satimage']
df_binary = df[df.dataset.isin(fsIs_b) | df.dataset.isin(fsIl_b) | df.dataset.isin(flIs_b) | df.dataset.isin(flIl_b)]
df_multi = df[df.dataset.isin(fsIs_m) | df.dataset.isin(fsIl_m) | df.dataset.isin(flIs_m) | df.dataset.isin(flIl_m)]
df = df_binary
#df.drop(df[(df.methods=='fc') & ((df.time_budget==30) | (df.time_budget==10))].index, inplace=True)
#df.drop(df[(df.methods=='3c') & ((df.time_budget==60) | (df.time_budget==10))].index, inplace=True)
df.shape

(581, 4)

In [31]:
rp.summary_cont(df['f1score'])

,Variable,N,Mean,SD,SE,95% Conf.,Interval
0,f1score,581.0,0.818827,0.20872,0.008659,0.80182,0.835834


In [32]:
rp.summary_cont(df.groupby(['time_budget']))['f1score']

,N,Mean,SD,SE,95% Conf.,Interval
time_budget,,,,,,
10,161,0.823724,0.197346,0.015553,0.793240,0.854208
30,223,0.828943,0.192694,0.012904,0.803652,0.854235
60,197,0.803375,0.233856,0.016662,0.770718,0.836031


In [33]:
rp.summary_cont(df.groupby(['methods']))['f1score']

,N,Mean,SD,SE,95% Conf.,Interval
methods,,,,,,
3c,63,0.841806,0.215475,0.027147,0.788597,0.895015
def,70,0.851325,0.176365,0.021080,0.810009,0.892641
dt,108,0.760173,0.222984,0.021457,0.718118,0.802228
fc,131,0.832187,0.204831,0.017896,0.797110,0.867264
rf,123,0.823052,0.207812,0.018738,0.786326,0.859778
svm,86,0.822810,0.208451,0.022478,0.778754,0.866867


In [34]:
rp.summary_cont(df.groupby(['time_budget', 'methods']))['f1score']

N      Mean        SD        SE  95% Conf.  Interval
time_budget methods                                                       
10          3c       12  0.837359  0.193220  0.055778   0.728034  0.946683
            def      20  0.872689  0.142873  0.031947   0.810073  0.935306
            dt       36  0.753362  0.226889  0.037815   0.679245  0.827479
            fc       46  0.870507  0.150621  0.022208   0.826979  0.914034
            rf       29  0.824350  0.202417  0.037588   0.750677  0.898022
            svm      18  0.780389  0.254371  0.059956   0.662875  0.897902
30          3c       23  0.870051  0.149337  0.031139   0.809018  0.931083
            def      21  0.873640  0.114389  0.024962   0.824715  0.922565
            dt       36  0.768883  0.203711  0.033952   0.702337  0.835428
            fc       52  0.814532  0.219875  0.030491   0.754769  0.874295
            rf       60  0.835738  0.203974  0.026333   0.784125  0.887350
            svm      31  0.848937  0.171401  0.030785   0.788599  0.909275
60          3c       28  0.820511  0.268469  0.050736   0.721068  0.919953
            def      29  0.820431  0.227518  0.042249   0.737623  0.903239
            dt       36  0.758273  0.242693  0.040449   0.678994  0.837553
            fc       33  0.806592  0.241245  0.041995   0.724281  0.888903
            rf       34  0.799557  0.222807  0.038211   0.724664  0.874451
            svm      37  0.821558  0.214636  0.035286   0.752398  0.890718

In [35]:
    # Fits the model with the interaction term
    # This will also automatically include the main effects for each factor
    model = ols('f1score ~ C(time_budget)*C(methods)', df).fit()

    # Seeing if the overall model is significant
    print(f"Overall model F({model.df_model: .0f},{model.df_resid: .0f}) = {model.fvalue: .3f}, p = {model.f_pvalue: .4f}")

Overall model F( 17, 563) =  1.050, p =  0.4016


In [36]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                f1score   R-squared:                       0.031
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.050
Date:                Fri, 01 Nov 2019   Prob (F-statistic):              0.402
Time:                        17:14:18   Log-Likelihood:                 95.451
No. Observations:                 581   AIC:                            -154.9
Df Residuals:                     563   BIC:                            -76.34
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
==========================================================================================================
                                             coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------
Intercept                                  0.8374      0.060     13.908      0.000       0.719       0.956
C(time_budget)[T.30]                       0.0327      0.074      0.440      0.660      -0.113       0.179
C(time_budget)[T.60]                      -0.0168      0.072     -0.234      0.815      -0.158       0.125
C(methods)[T.def]                          0.0353      0.076      0.464      0.643      -0.114       0.185
C(methods)[T.dt]                          -0.0840      0.070     -1.208      0.227      -0.221       0.053
C(methods)[T.fc]                           0.0331      0.068      0.490      0.624      -0.100       0.166
C(methods)[T.rf]                          -0.0130      0.072     -0.182      0.856      -0.154       0.128
C(methods)[T.svm]                         -0.0570      0.078     -0.733      0.464      -0.210       0.096
C(time_budget)[T.30]:C(methods)[T.def]    -0.0317      0.099     -0.321      0.748      -0.226       0.162
C(time_budget)[T.60]:C(methods)[T.def]    -0.0354      0.094     -0.376      0.707      -0.220       0.149
C(time_budget)[T.30]:C(methods)[T.dt]     -0.0172      0.089     -0.193      0.847      -0.192       0.158
C(time_budget)[T.60]:C(methods)[T.dt]      0.0218      0.087      0.250      0.803      -0.149       0.193
C(time_budget)[T.30]:C(methods)[T.fc]     -0.0887      0.085     -1.038      0.300      -0.256       0.079
C(time_budget)[T.60]:C(methods)[T.fc]     -0.0471      0.086     -0.546      0.586      -0.217       0.122
C(time_budget)[T.30]:C(methods)[T.rf]     -0.0213      0.088     -0.242      0.809      -0.194       0.152
C(time_budget)[T.60]:C(methods)[T.rf]     -0.0079      0.089     -0.089      0.929      -0.183       0.167
C(time_budget)[T.30]:C(methods)[T.svm]     0.0359      0.097      0.371      0.711      -0.154       0.226
C(time_budget)[T.60]:C(methods)[T.svm]     0.0580      0.094      0.619      0.536      -0.126       0.242
==============================================================================
Omnibus:                      161.222   Durbin-Watson:                   2.238
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              320.292
Skew:                          -1.580   Prob(JB):                     2.82e-70
Kurtosis:                       4.803   Cond. No.                         39.4
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [37]:
# Creates the ANOVA table
res = sm.stats.anova_lm(model, typ= 2)
res

,sum_sq,df,F,PR(>F)
C(time_budget),0.083640,2.0,0.961368,0.382996
C(methods),0.515610,5.0,2.370587,0.038213
C(time_budget):C(methods),0.186896,10.0,0.429640,0.932258
Residual,24.490842,563.0,NaN,NaN


In [38]:
# Fits the model
model2 = ols('f1score ~ C(time_budget)+ C(methods)', df).fit()

print(f"Overall model F({model2.df_model: .0f},{model2.df_resid: .0f}) = {model2.fvalue: .3f}, p = {model2.f_pvalue: .4f}")

Overall model F( 7, 573) =  1.955, p =  0.0591


In [39]:
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                f1score   R-squared:                       0.023
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     1.955
Date:                Fri, 01 Nov 2019   Prob (F-statistic):             0.0591
Time:                        17:14:19   Log-Likelihood:                 93.242
No. Observations:                 581   AIC:                            -170.5
Df Residuals:                     573   BIC:                            -135.6
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                0.8519      0.031     27.895      0.000       0.792       0.912
C(time_budget)[T.30]     0.0021      0.022      0.096      0.924      -0.040       0.045
C(time_budget)[T.60]    -0.0244      0.022     -1.093      0.275      -0.068       0.019
C(methods)[T.def]        0.0089      0.036      0.247      0.805      -0.062       0.080
C(methods)[T.dt]        -0.0843      0.033     -2.552      0.011      -0.149      -0.019
C(methods)[T.fc]        -0.0144      0.032     -0.449      0.654      -0.077       0.049
C(methods)[T.rf]        -0.0231      0.032     -0.715      0.475      -0.087       0.040
C(methods)[T.svm]       -0.0193      0.034     -0.562      0.575      -0.087       0.048
==============================================================================
Omnibus:                      166.887   Durbin-Watson:                   2.220
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              341.519
Skew:                          -1.613   Prob(JB):                     6.92e-75
Kurtosis:                       4.923   Cond. No.                         9.49
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [40]:
# Creates the ANOVA table
res2 = sm.stats.anova_lm(model2, typ= 2)
res2

,sum_sq,df,F,PR(>F)
C(time_budget),0.083640,2.0,0.971033,0.379314
C(methods),0.515610,5.0,2.394421,0.036474
Residual,24.677738,573.0,NaN,NaN


In [41]:
# Calculating effect size
def anova_table(aov):
    aov['mean_sq'] = aov[:]['sum_sq']/aov[:]['df']
    
    aov['eta_sq'] = aov[:-1]['sum_sq']/sum(aov['sum_sq'])
    
    aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df']*aov['mean_sq'][-1]))/(sum(aov['sum_sq'])+aov['mean_sq'][-1])
    
    cols = ['sum_sq', 'mean_sq', 'df', 'F', 'PR(>F)', 'eta_sq', 'omega_sq']
    aov = aov[cols]
    return aov

anova_table(res2)

,sum_sq,mean_sq,df,F,PR(>F),eta_sq,omega_sq
C(time_budget),0.083640,0.041820,2.0,0.971033,0.379314,0.003309,-0.000099
C(methods),0.515610,0.103122,5.0,2.394421,0.036474,0.020398,0.011859
Residual,24.677738,0.043068,573.0,NaN,NaN,NaN,NaN


In [42]:
mc = statsmodels.stats.multicomp.MultiComparison(df['f1score'], df['methods'])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
    3c    def   0.0095    0.9 -0.0935  0.1126  False
    3c     dt  -0.0816  0.131 -0.1757  0.0124  False
    3c     fc  -0.0096    0.9 -0.1006  0.0814  False
    3c     rf  -0.0188    0.9 -0.1107  0.0732  False
    3c    svm   -0.019    0.9 -0.1174  0.0794  False
   def     dt  -0.0912 0.0495 -0.1822 -0.0001   True
   def     fc  -0.0191    0.9  -0.107  0.0687  False
   def     rf  -0.0283    0.9 -0.1171  0.0606  False
   def    svm  -0.0285    0.9  -0.124   0.067  False
    dt     fc    0.072 0.0832 -0.0051  0.1491  False
    dt     rf   0.0629 0.1965 -0.0154  0.1411  False
    dt    svm   0.0626 0.2948 -0.0231  0.1484  False
    fc     rf  -0.0091    0.9 -0.0836  0.0654  False
    fc    svm  -0.0094    0.9 -0.0917   0.073  False
    rf    svm  -0.0002    0.9 -0.0836  0.0832  False
----------------------------------------------